# Importações

In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, DateTime, String, Float
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

#  PostgreSQL

### 1. Conexão 

In [4]:
db_config = {
    'bucket': 'Seasonality',
    'user': 'avnadmin',
    'password': 'AVNS_9fZb3BkX9qGXxKpxsrZ',
    'host': 'postgresql-iscac.f.aivencloud.com',  
    'port': 25674,
    'table': 'meteorology' 
}

db_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['bucket']}"
table_name = db_config['table']

In [5]:
csv_file = "CSV/lisbon_2024_12_13_to_2024_12_31.csv"

### 2. Definição do modelo de dados

In [6]:
# Base para definição dos modelos
Base = declarative_base()

# Definir o modelo para a tabela
class Meteorology(Base):
    __tablename__ = table_name
    datetime = Column(DateTime, primary_key=True)
    temp = Column(Float)
    feelslike = Column(Float)
    dew = Column(Float)
    humidity = Column(Float)
    precip = Column(Float)
    precipprob = Column(Float)
    preciptype = Column(String)
    windgust = Column(Float)
    windspeed = Column(Float)
    winddir = Column(Float)
    sealevelpressure = Column(Float)
    cloudcover = Column(Float)
    visibility = Column(Float)
    conditions = Column(String)
    stations = Column(String)

### 3. Guardar CSV no PostgreSql

In [8]:
engine = create_engine(db_url)

def import_csv_to_postgres(csv_file, table_name):
    # Carregar os dados do CSV usando pandas
    df = pd.read_csv(csv_file)
    
    # Remover colunas sem valores
    df = df.drop(columns=['name', 'icon', 'snow', 'snowdepth', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk'])
    
    # Substituir NaN por None (NULL no PostgreSQL)
    df = df.where(pd.notna(df), None)
    
    # Criar as tabelas no banco de dados com a chave primária definida
    Base.metadata.create_all(engine)
    
    # Inserir os dados no banco de dados, criando a tabela automaticamente
    df.to_sql(table_name, engine, if_exists='append', index=False)
    print(f"Dados do CSV importados para a tabela {table_name} com sucesso!")

def main():
    try:
        import_csv_to_postgres(csv_file, table_name)
    except Exception as e:
        print(f"Erro ao importar os dados: {e}")

if __name__ == "__main__":
    main()


Dados do CSV importados para a tabela meteorology com sucesso!
